In [1]:
import keras

Using TensorFlow backend.


In [2]:
keras.__version__

'1.2.0'

In [1]:
#pip install keras==1.2.2
import keras.backend as K
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout,  GaussianNoise,BatchNormalization, Activation, Embedding, ZeroPadding2D,MaxPooling2D, merge #,multiply
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
#from keras import losses
#from keras.utils import to_categorical
from __future__ import print_function, division
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [24]:
global dim_x # x-dimension of the image or the number of rows
dim_x = 28   
global dim_y  # y-dimension of the image or the number of columns
dim_y =  28
global n_channels    # number of channels in the image
n_channels = 1

In [25]:
class AdversarialAutoencoder():
    
    # Initializer Class
    def __init__(self):
        self.img_rows = dim_x
        self.img_cols = dim_y
        self.channels = n_channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 10

        optimizer = Adam(0.0002, 0.5) # 0.0002 is the learning rate and 0.5 is the Beta

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

        # Build the encoder / decoder
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()

        img = Input(shape=self.img_shape)
        # The generator takes the image, encodes it and reconstructs it
        # from the encoding
        encoded_repr = self.encoder(img)
        reconstructed_img = self.decoder(encoded_repr)

        # For the adversarial_autoencoder model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator determines validity of the encoding
        validity = self.discriminator(encoded_repr)

        # The adversarial_autoencoder model  (stacked generator and discriminator)
        self.adversarial_autoencoder = Model(img, [reconstructed_img, validity])
        self.adversarial_autoencoder.compile(loss=['mse', 'binary_crossentropy'],
            loss_weights=[0.999, 0.001],
            optimizer=optimizer)


    #------------------------------------------------------------------------------------------------
    # Encoder Class
    def build_encoder(self):
        # Encoder

        img = Input(shape=self.img_shape)

        #Coding of the model starts here
        
        #Structure of the network is: 

        # Flatten the input --> Block of (Fully Connected with  512 output nodes --> LeakyReLU) repeated 2 times --> 
        # 2 Fully Connected Layer corresponding to 'mu' and 'sigma' of distribution with output nodes = latent dimension (this is the bottleneck State of our encoder) -->
        # Merging the 'mu' and 'sigma' layer with 
        
        h = Flatten()(img)
        h = Dense(512)(h)
        h = LeakyReLU(alpha=0.2)(h)
        h = Dense(512)(h)
        h = LeakyReLU(alpha=0.2)(h)
        mu = Dense(self.latent_dim)(h)
        log_var = Dense(self.latent_dim)(h)
        latent_repr = merge([mu, log_var],mode=lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2),output_shape=lambda p: p[0])
        return Model(img, latent_repr)

    #-----------------------------------------------------------------------------------------------------
    # Decoder Class
    def build_decoder(self):

        # Structure of the decoder is
        # Input latent layer with latent_dim --> add a leaky Relu --> add a dense layer (repeat 2 times)
        # --> Reshape the last dense layer to image size

        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        z = Input(shape=(self.latent_dim,))
        img = model(z)

        return Model(z, img)


    # Discriminator Class-----------------------------------------------------------------
    def build_discriminator(self):

    	# Discriminator class
    	# Structure 
    	# Take input from latent layer with latent dim --> add leaky relu --> dense layer --> leaky relu
    	# dense 


        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation="sigmoid"))
        model.summary()

        encoded_repr = Input(shape=(self.latent_dim, ))
        validity = model(encoded_repr)

        return Model(encoded_repr, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            latent_fake = self.encoder.predict(imgs)
            latent_real = np.random.normal(size=(batch_size, self.latent_dim))

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(latent_real, valid)
            d_loss_fake = self.discriminator.train_on_batch(latent_fake, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.adversarial_autoencoder.train_on_batch(imgs, [imgs, valid])

            # Plot the progress
            print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5

        z = np.random.normal(size=(r*c, self.latent_dim))
        gen_imgs = self.decoder.predict(z)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "aae_generator")
        save(self.discriminator, "aae_discriminator")


In [26]:
aae = AdversarialAutoencoder()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_21 (Dense)                 (None, 512)           5632        dense_input_5[0][0]              
____________________________________________________________________________________________________
leakyrelu_13 (LeakyReLU)         (None, 512)           0           dense_21[0][0]                   
____________________________________________________________________________________________________
dense_22 (Dense)                 (None, 256)           131328      leakyrelu_13[0][0]               
____________________________________________________________________________________________________
leakyrelu_14 (LeakyReLU)         (None, 256)           0           dense_22[0][0]                   
___________________________________________________________________________________________

In [27]:
aae.train(epochs=20000, batch_size=32, sample_interval=200)

0 [D loss: 0.743175, acc: 50.00%] [G loss: 0.938738, mse: 0.939085]
1 [D loss: 0.678181, acc: 50.00%] [G loss: 0.924853, mse: 0.925125]
2 [D loss: 0.639302, acc: 79.69%] [G loss: 0.904661, mse: 0.904801]
3 [D loss: 0.541667, acc: 95.31%] [G loss: 0.864548, mse: 0.864288]
4 [D loss: 0.386443, acc: 93.75%] [G loss: 0.782866, mse: 0.781619]
5 [D loss: 0.322697, acc: 89.06%] [G loss: 0.670551, mse: 0.667630]
6 [D loss: 0.271591, acc: 92.19%] [G loss: 0.546698, mse: 0.542092]
7 [D loss: 0.302970, acc: 89.06%] [G loss: 0.421720, mse: 0.415779]
8 [D loss: 0.260205, acc: 93.75%] [G loss: 0.381267, mse: 0.374852]
9 [D loss: 0.242387, acc: 95.31%] [G loss: 0.338169, mse: 0.331318]
10 [D loss: 0.228332, acc: 93.75%] [G loss: 0.310734, mse: 0.304040]
11 [D loss: 0.242744, acc: 95.31%] [G loss: 0.263576, mse: 0.256932]
12 [D loss: 0.238623, acc: 95.31%] [G loss: 0.284399, mse: 0.278603]
13 [D loss: 0.187380, acc: 100.00%] [G loss: 0.248417, mse: 0.242915]
14 [D loss: 0.202881, acc: 95.31%] [G loss:

125 [D loss: 0.320066, acc: 85.94%] [G loss: 0.156692, mse: 0.151563]
126 [D loss: 0.266075, acc: 85.94%] [G loss: 0.147236, mse: 0.139725]
127 [D loss: 0.277965, acc: 89.06%] [G loss: 0.153647, mse: 0.148702]
128 [D loss: 0.254604, acc: 85.94%] [G loss: 0.138786, mse: 0.133781]
129 [D loss: 0.282005, acc: 87.50%] [G loss: 0.156914, mse: 0.151313]
130 [D loss: 0.383456, acc: 84.38%] [G loss: 0.148276, mse: 0.144191]
131 [D loss: 0.217282, acc: 92.19%] [G loss: 0.155396, mse: 0.147384]
132 [D loss: 0.308543, acc: 84.38%] [G loss: 0.147181, mse: 0.141427]
133 [D loss: 0.323057, acc: 82.81%] [G loss: 0.138814, mse: 0.133499]
134 [D loss: 0.275090, acc: 85.94%] [G loss: 0.147643, mse: 0.142443]
135 [D loss: 0.324716, acc: 79.69%] [G loss: 0.157681, mse: 0.152675]
136 [D loss: 0.218823, acc: 92.19%] [G loss: 0.129503, mse: 0.124469]
137 [D loss: 0.232693, acc: 90.62%] [G loss: 0.129482, mse: 0.124554]
138 [D loss: 0.218566, acc: 98.44%] [G loss: 0.147579, mse: 0.143264]
139 [D loss: 0.20524

243 [D loss: 0.052401, acc: 100.00%] [G loss: 0.133664, mse: 0.128179]
244 [D loss: 0.055898, acc: 100.00%] [G loss: 0.110625, mse: 0.105158]
245 [D loss: 0.049481, acc: 100.00%] [G loss: 0.126234, mse: 0.120963]
246 [D loss: 0.067801, acc: 100.00%] [G loss: 0.117051, mse: 0.111613]
247 [D loss: 0.055107, acc: 100.00%] [G loss: 0.115660, mse: 0.110148]
248 [D loss: 0.064639, acc: 100.00%] [G loss: 0.138127, mse: 0.132719]
249 [D loss: 0.073021, acc: 100.00%] [G loss: 0.126722, mse: 0.121355]
250 [D loss: 0.048865, acc: 100.00%] [G loss: 0.119231, mse: 0.113461]
251 [D loss: 0.051257, acc: 100.00%] [G loss: 0.124311, mse: 0.118920]
252 [D loss: 0.055210, acc: 100.00%] [G loss: 0.114140, mse: 0.108787]
253 [D loss: 0.063484, acc: 100.00%] [G loss: 0.125976, mse: 0.121343]
254 [D loss: 0.045948, acc: 100.00%] [G loss: 0.094355, mse: 0.088704]
255 [D loss: 0.048580, acc: 100.00%] [G loss: 0.125167, mse: 0.119653]
256 [D loss: 0.055337, acc: 100.00%] [G loss: 0.117156, mse: 0.111677]
257 [D

364 [D loss: 0.018018, acc: 100.00%] [G loss: 0.100719, mse: 0.094105]
365 [D loss: 0.021986, acc: 100.00%] [G loss: 0.122234, mse: 0.116166]
366 [D loss: 0.024374, acc: 100.00%] [G loss: 0.104037, mse: 0.097993]
367 [D loss: 0.019421, acc: 100.00%] [G loss: 0.101676, mse: 0.095197]
368 [D loss: 0.022053, acc: 100.00%] [G loss: 0.120756, mse: 0.114814]
369 [D loss: 0.021697, acc: 100.00%] [G loss: 0.113628, mse: 0.106921]
370 [D loss: 0.018533, acc: 100.00%] [G loss: 0.108821, mse: 0.102589]
371 [D loss: 0.017026, acc: 100.00%] [G loss: 0.103300, mse: 0.097751]
372 [D loss: 0.028304, acc: 100.00%] [G loss: 0.128925, mse: 0.123549]
373 [D loss: 0.016623, acc: 100.00%] [G loss: 0.099247, mse: 0.093459]
374 [D loss: 0.020799, acc: 100.00%] [G loss: 0.109766, mse: 0.103280]
375 [D loss: 0.018990, acc: 100.00%] [G loss: 0.097682, mse: 0.091766]
376 [D loss: 0.022083, acc: 100.00%] [G loss: 0.094954, mse: 0.089114]
377 [D loss: 0.023383, acc: 100.00%] [G loss: 0.125019, mse: 0.118744]
378 [D

482 [D loss: 0.010486, acc: 100.00%] [G loss: 0.108811, mse: 0.101754]
483 [D loss: 0.012231, acc: 100.00%] [G loss: 0.110674, mse: 0.104567]
484 [D loss: 0.012443, acc: 100.00%] [G loss: 0.100156, mse: 0.093705]
485 [D loss: 0.010688, acc: 100.00%] [G loss: 0.116727, mse: 0.109801]
486 [D loss: 0.007983, acc: 100.00%] [G loss: 0.107686, mse: 0.100833]
487 [D loss: 0.011637, acc: 100.00%] [G loss: 0.105853, mse: 0.099286]
488 [D loss: 0.020212, acc: 100.00%] [G loss: 0.113936, mse: 0.107907]
489 [D loss: 0.012904, acc: 100.00%] [G loss: 0.100298, mse: 0.094151]
490 [D loss: 0.019233, acc: 100.00%] [G loss: 0.117781, mse: 0.111441]
491 [D loss: 0.014384, acc: 100.00%] [G loss: 0.092172, mse: 0.085461]
492 [D loss: 0.013636, acc: 100.00%] [G loss: 0.111975, mse: 0.105136]
493 [D loss: 0.010505, acc: 100.00%] [G loss: 0.128069, mse: 0.121424]
494 [D loss: 0.029904, acc: 100.00%] [G loss: 0.114264, mse: 0.107525]
495 [D loss: 0.012782, acc: 100.00%] [G loss: 0.098684, mse: 0.091764]
496 [D

...

19419 [D loss: 0.562475, acc: 70.31%] [G loss: 0.068987, mse: 0.067454]
19420 [D loss: 0.331424, acc: 85.94%] [G loss: 0.049780, mse: 0.047664]
19421 [D loss: 0.330965, acc: 85.94%] [G loss: 0.057610, mse: 0.055691]
19422 [D loss: 0.484763, acc: 79.69%] [G loss: 0.059309, mse: 0.057782]
19423 [D loss: 0.336422, acc: 87.50%] [G loss: 0.053372, mse: 0.051356]
19424 [D loss: 0.387605, acc: 82.81%] [G loss: 0.057785, mse: 0.055700]
19425 [D loss: 0.445918, acc: 79.69%] [G loss: 0.057161, mse: 0.055439]
19426 [D loss: 0.631324, acc: 73.44%] [G loss: 0.064553, mse: 0.062925]
19427 [D loss: 0.377649, acc: 84.38%] [G loss: 0.058813, mse: 0.057215]
19428 [D loss: 0.469594, acc: 78.12%] [G loss: 0.062869, mse: 0.061401]
19429 [D loss: 0.668295, acc: 65.62%] [G loss: 0.069921, mse: 0.068328]
19430 [D loss: 0.450224, acc: 78.12%] [G loss: 0.051591, mse: 0.049853]
19431 [D loss: 0.305054, acc: 87.50%] [G loss: 0.054837, mse: 0.052818]
19432 [D loss: 0.420620, acc: 81.25%] [G loss: 0.056565, mse: 0.

19539 [D loss: 0.404179, acc: 79.69%] [G loss: 0.055916, mse: 0.054111]
19540 [D loss: 0.389852, acc: 79.69%] [G loss: 0.061293, mse: 0.059400]
19541 [D loss: 0.445466, acc: 82.81%] [G loss: 0.055789, mse: 0.053779]
19542 [D loss: 0.433639, acc: 79.69%] [G loss: 0.056692, mse: 0.055063]
19543 [D loss: 0.470705, acc: 79.69%] [G loss: 0.055417, mse: 0.053474]
19544 [D loss: 0.395202, acc: 82.81%] [G loss: 0.059710, mse: 0.057726]
19545 [D loss: 0.377482, acc: 79.69%] [G loss: 0.045408, mse: 0.043642]
19546 [D loss: 0.323000, acc: 85.94%] [G loss: 0.056351, mse: 0.054462]
19547 [D loss: 0.396265, acc: 85.94%] [G loss: 0.054862, mse: 0.052953]
19548 [D loss: 0.272044, acc: 92.19%] [G loss: 0.047414, mse: 0.045060]
19549 [D loss: 0.468090, acc: 76.56%] [G loss: 0.058128, mse: 0.056858]
19550 [D loss: 0.337560, acc: 85.94%] [G loss: 0.053659, mse: 0.051899]
19551 [D loss: 0.414822, acc: 85.94%] [G loss: 0.054119, mse: 0.052077]
19552 [D loss: 0.399272, acc: 82.81%] [G loss: 0.054624, mse: 0.

19658 [D loss: 0.387114, acc: 79.69%] [G loss: 0.054064, mse: 0.051670]
19659 [D loss: 0.453008, acc: 78.12%] [G loss: 0.056794, mse: 0.054860]
19660 [D loss: 0.321281, acc: 85.94%] [G loss: 0.055218, mse: 0.053063]
19661 [D loss: 0.378044, acc: 84.38%] [G loss: 0.050484, mse: 0.048519]
19662 [D loss: 0.511643, acc: 71.88%] [G loss: 0.057252, mse: 0.055420]
19663 [D loss: 0.472275, acc: 79.69%] [G loss: 0.062336, mse: 0.060566]
19664 [D loss: 0.497864, acc: 78.12%] [G loss: 0.065346, mse: 0.063469]
19665 [D loss: 0.376743, acc: 81.25%] [G loss: 0.050488, mse: 0.048524]
19666 [D loss: 0.503802, acc: 73.44%] [G loss: 0.052631, mse: 0.050601]
19667 [D loss: 0.360646, acc: 85.94%] [G loss: 0.056785, mse: 0.054857]
19668 [D loss: 0.398713, acc: 79.69%] [G loss: 0.060080, mse: 0.058201]
19669 [D loss: 0.346444, acc: 84.38%] [G loss: 0.055947, mse: 0.054164]
19670 [D loss: 0.271324, acc: 85.94%] [G loss: 0.057342, mse: 0.055268]
19671 [D loss: 0.531146, acc: 75.00%] [G loss: 0.045125, mse: 0.

19775 [D loss: 0.322775, acc: 89.06%] [G loss: 0.058126, mse: 0.056117]
19776 [D loss: 0.502461, acc: 78.12%] [G loss: 0.058132, mse: 0.056529]
19777 [D loss: 0.345964, acc: 87.50%] [G loss: 0.067773, mse: 0.066040]
19778 [D loss: 0.421537, acc: 79.69%] [G loss: 0.044662, mse: 0.042463]
19779 [D loss: 0.401002, acc: 84.38%] [G loss: 0.061813, mse: 0.060143]
19780 [D loss: 0.281647, acc: 90.62%] [G loss: 0.052570, mse: 0.050491]
19781 [D loss: 0.392298, acc: 85.94%] [G loss: 0.062508, mse: 0.060811]
19782 [D loss: 0.426827, acc: 81.25%] [G loss: 0.046577, mse: 0.044641]
19783 [D loss: 0.352957, acc: 81.25%] [G loss: 0.049611, mse: 0.047670]
19784 [D loss: 0.505906, acc: 81.25%] [G loss: 0.068048, mse: 0.066306]
19785 [D loss: 0.317652, acc: 87.50%] [G loss: 0.053074, mse: 0.050639]
19786 [D loss: 0.440285, acc: 78.12%] [G loss: 0.056792, mse: 0.054803]
19787 [D loss: 0.485988, acc: 75.00%] [G loss: 0.062400, mse: 0.060534]
19788 [D loss: 0.401231, acc: 84.38%] [G loss: 0.052507, mse: 0.

19893 [D loss: 0.349991, acc: 84.38%] [G loss: 0.058318, mse: 0.056258]
19894 [D loss: 0.384149, acc: 82.81%] [G loss: 0.053629, mse: 0.051800]
19895 [D loss: 0.267159, acc: 85.94%] [G loss: 0.056382, mse: 0.054278]
19896 [D loss: 0.403179, acc: 81.25%] [G loss: 0.056884, mse: 0.055101]
19897 [D loss: 0.491954, acc: 82.81%] [G loss: 0.060723, mse: 0.058840]
19898 [D loss: 0.363958, acc: 84.38%] [G loss: 0.043092, mse: 0.041032]
19899 [D loss: 0.425564, acc: 79.69%] [G loss: 0.057799, mse: 0.055981]
19900 [D loss: 0.507258, acc: 78.12%] [G loss: 0.057655, mse: 0.055750]
19901 [D loss: 0.468700, acc: 78.12%] [G loss: 0.054644, mse: 0.052673]
19902 [D loss: 0.412130, acc: 76.56%] [G loss: 0.051430, mse: 0.049890]
19903 [D loss: 0.462917, acc: 75.00%] [G loss: 0.063626, mse: 0.062032]
19904 [D loss: 0.326327, acc: 92.19%] [G loss: 0.054673, mse: 0.052431]
19905 [D loss: 0.386574, acc: 82.81%] [G loss: 0.053638, mse: 0.051666]
19906 [D loss: 0.416086, acc: 89.06%] [G loss: 0.053456, mse: 0.

In [6]:
from keras.datasets import mnist

In [7]:
(X_train, _), (_, _) = mnist.load_data()

In [8]:
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

In [9]:
import numpy as np

In [11]:
X_train.shape[0]

60000

In [12]:
idx = np.random.randint(0, X_train.shape[0], 32)

In [17]:
idx

array([49602, 10851, 28153, 36532, 14387, 55986, 38936, 59561, 59210,
       10702, 36137,  6823, 38386, 41140, 30122,  4090, 46512, 40887,
       32889, 16504, 27838, 52121, 33493, 42704, 18498, 11062,  4368,
       35816, 10439, 50572, 20029, 47615])

In [18]:
img = X_train[idx]

In [19]:
img.shape

(32, 28, 28, 1)